# Project 1b: Lasso regression
**Author: Chenwei Xiao**

**Date: 2020/4**

## Task1b:
* Extent the features from x1-5 to also squared,exponential,cosine
* Find best regression model with cross validation

In [2]:
# IPython Commands.
%matplotlib inline
# %reload_ext autoreload
# %load_ext autoreload
# %autoreload 2

# General math and plotting modules.
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt


# Machine Learning library. 
import sklearn
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from sklearn.linear_model import Lasso, LassoCV
from sklearn.model_selection import KFold

In [3]:
# Load data
df_train = pd.read_csv('train.csv')
print(df_train.shape)
df_train.head()

(700, 7)


,Id,y,x1,x2,x3,x4,x5
0,0,3.579620,0.02,0.05,-0.09,-0.43,-0.08
1,1,0.734869,-0.13,0.11,-0.08,-0.29,-0.03
2,2,4.287433,0.08,0.06,-0.07,-0.41,-0.03
3,3,8.347827,0.02,-0.12,0.01,-0.43,-0.02
4,4,-0.052718,-0.14,-0.12,-0.08,-0.02,-0.08


In [4]:
X = df_train.iloc[:,2:]
y = df_train.iloc[:,1]
# Create more features
X_extend = pd.concat([X, X**2,np.cos(X),np.exp(X)], axis=1, ignore_index=True)
X_extend

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.02,0.05,-0.09,-0.43,-0.08,0.0004,0.0025,0.0081,0.1849,0.0064,0.999800,0.998750,0.995953,0.908966,0.996802,1.020201,1.051271,0.913931,0.650509,0.923116
1,-0.13,0.11,-0.08,-0.29,-0.03,0.0169,0.0121,0.0064,0.0841,0.0009,0.991562,0.993956,0.996802,0.958244,0.999550,0.878095,1.116278,0.923116,0.748264,0.970446
2,0.08,0.06,-0.07,-0.41,-0.03,0.0064,0.0036,0.0049,0.1681,0.0009,0.996802,0.998201,0.997551,0.917121,0.999550,1.083287,1.061837,0.932394,0.663650,0.970446
3,0.02,-0.12,0.01,-0.43,-0.02,0.0004,0.0144,0.0001,0.1849,0.0004,0.999800,0.992809,0.999950,0.908966,0.999800,1.020201,0.886920,1.010050,0.650509,0.980199
4,-0.14,-0.12,-0.08,-0.02,-0.08,0.0196,0.0144,0.0064,0.0004,0.0064,0.990216,0.992809,0.996802,0.999800,0.996802,0.869358,0.886920,0.923116,0.980199,0.923116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,0.31,-0.09,0.04,-0.09,0.03,0.0961,0.0081,0.0016,0.0081,0.0009,0.952334,0.995953,0.999200,0.995953,0.999550,1.363425,0.913931,1.040811,0.913931,1.030455
696,-0.26,-0.01,0.02,-0.40,0.05,0.0676,0.0001,0.0004,0.1600,0.0025,0.966390,0.999950,0.999800,0.921061,0.998750,0.771052,0.990050,1.020201,0.670320,1.051271
697,-0.27,-0.22,-0.01,-0.32,-0.05,0.0729,0.0484,0.0001,0.1024,0.0025,0.963771,0.975897,0.999950,0.949235,0.998750,0.763379,0.802519,0.990050,0.726149,0.951229
698,0.19,0.11,-0.05,-0.27,-0.04,0.0361,0.0121,0.0025,0.0729,0.0016,0.982004,0.993956,0.998750,0.963771,0.999200,1.209250,1.116278,0.951229,0.763379,0.960789


In [5]:
## shuffle dataset
y,X_extend = shuffle(y,X_extend,random_state=42)
X_extend

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
158,-0.07,-0.09,-0.03,-0.40,-0.08,0.0049,0.0081,0.0009,0.1600,0.0064,0.997551,0.995953,0.999550,0.921061,0.996802,0.932394,0.913931,0.970446,0.670320,0.923116
500,-0.13,-0.04,-0.03,-0.04,-0.07,0.0169,0.0016,0.0009,0.0016,0.0049,0.991562,0.999200,0.999550,0.999200,0.997551,0.878095,0.960789,0.970446,0.960789,0.932394
396,-0.35,-0.18,-0.07,-0.40,-0.05,0.1225,0.0324,0.0049,0.1600,0.0025,0.939373,0.983844,0.997551,0.921061,0.998750,0.704688,0.835270,0.932394,0.670320,0.951229
155,-0.14,-0.25,-0.06,-0.41,-0.03,0.0196,0.0625,0.0036,0.1681,0.0009,0.990216,0.968912,0.998201,0.917121,0.999550,0.869358,0.778801,0.941765,0.663650,0.970446
321,-0.21,-0.20,0.01,-0.39,-0.04,0.0441,0.0400,0.0001,0.1521,0.0016,0.978031,0.980067,0.999950,0.924909,0.999200,0.810584,0.818731,1.010050,0.677057,0.960789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,0.02,-0.05,-0.07,-0.29,0.01,0.0004,0.0025,0.0049,0.0841,0.0001,0.999800,0.998750,0.997551,0.958244,0.999950,1.020201,0.951229,0.932394,0.748264,1.010050
106,-0.14,-0.21,-0.02,-0.38,0.00,0.0196,0.0441,0.0004,0.1444,0.0000,0.990216,0.978031,0.999800,0.928665,1.000000,0.869358,0.810584,0.980199,0.683861,1.000000
270,-0.10,-0.21,-0.03,-0.40,-0.05,0.0100,0.0441,0.0009,0.1600,0.0025,0.995004,0.978031,0.999550,0.921061,0.998750,0.904837,0.810584,0.970446,0.670320,0.951229
435,-0.12,-0.03,-0.04,-0.19,-0.06,0.0144,0.0009,0.0016,0.0361,0.0036,0.992809,0.999550,0.999200,0.982004,0.998201,0.886920,0.970446,0.960789,0.826959,0.941765


In [26]:
alphas = np.logspace(-4, -0.5, 30)
alphas

array([1.00000000e-04, 1.32035178e-04, 1.74332882e-04, 2.30180731e-04,
       3.03919538e-04, 4.01280703e-04, 5.29831691e-04, 6.99564216e-04,
       9.23670857e-04, 1.21957046e-03, 1.61026203e-03, 2.12611233e-03,
       2.80721620e-03, 3.70651291e-03, 4.89390092e-03, 6.46167079e-03,
       8.53167852e-03, 1.12648169e-02, 1.48735211e-02, 1.96382800e-02,
       2.59294380e-02, 3.42359796e-02, 4.52035366e-02, 5.96845700e-02,
       7.88046282e-02, 1.04049831e-01, 1.37382380e-01, 1.81393069e-01,
       2.39502662e-01, 3.16227766e-01])

In [27]:
X = np.array(X_extend)
y = np.array(y)
lasso_cv = LassoCV(alphas=alphas, random_state=0, max_iter=1000000)
k_fold = KFold(3)

print("Answer to the bonus question:",
      "how much can you trust the selection of alpha?")
print()
print("Alpha parameters maximising the generalization score on different")
print("subsets of the data:")
for k, (train, test) in enumerate(k_fold.split(X, y)):
    lasso_cv.fit(X[train,:], y[train])
    print("[fold {0}] alpha: {1:.5f}, score: {2:.5f}".
          format(k, lasso_cv.alpha_, lasso_cv.score(X[test,:], y[test])))
print()
print("Answer: Not very much since we obtained different alphas for different")
print("subsets of the data and moreover, the scores for these alphas differ")
print("quite substantially.")

Answer to the bonus question: how much can you trust the selection of alpha?

Alpha parameters maximising the generalization score on different
subsets of the data:
[fold 0] alpha: 0.31623, score: -0.00291


D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


[fold 1] alpha: 0.31623, score: -0.00455


D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


[fold 2] alpha: 0.00489, score: -0.10467

Answer: Not very much since we obtained different alphas for different
subsets of the data and moreover, the scores for these alphas differ
quite substantially.


In [31]:
Lassoreg = LassoCV(cv=3,max_iter=1e6, random_state=0).fit(X_extend, y)

In [32]:
Lassoreg.alpha_

0.06994347777325974

In [33]:
Lassoreg.intercept_

1.0033391596316639

In [34]:
Lassoreg.coef_

array([ 0.00000000e+00, -0.00000000e+00,  0.00000000e+00,  4.48003759e-16,
        0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
       -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00])